In [78]:
import pandas as pd
import re

# Load the data
df_leads = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/prospectos_2023.csv")

# Define columns and their target data types
columns_dtype = {
    "mes": "int64",
    "partida_arancelaria": "int64",
    "ruc_importador": "int64",
    "dau": "int64",
    "advalorem": "float64",
    "unidades": "float64",
    "peso_neto": "float64",
    "peso_bruto": "float64",
    "us$_fob": "float64",
    "us$_flete": "float64",
    "us$_seguro": "float64",
    "us$_cif": "float64",
    "bultos": "float64",
}

# Filter out columns that don't exist in the dataset before applying dtype conversion
existing_columns = {col: dtype for col, dtype in columns_dtype.items() if col in df_leads.columns}

# Convert the existing columns to the appropriate data types
df_leads = df_leads.astype(existing_columns)

# List of date columns
date_columns = ["fecha_de_embarque", "fecha_de_llegada", "fecha_ingreso_sistema", "fecha"]

def preprocess_dates(column):
    """Preprocess the date columns by stripping spaces and replacing underscores."""
    return (
        column.astype(str)
        .str.strip()  # Remove leading/trailing spaces
        .str.replace("_", " ")  # Replace underscores with spaces
        .str.split(" ", n=1, expand=True)[0]  # Keep only the date part before the time
    )

# Apply preprocessing to date columns if they exist in the dataset
for col in date_columns:
    if col in df_leads.columns:
        df_leads[col] = preprocess_dates(df_leads[col])

# Convert to datetime format for the date columns
for col in date_columns:
    if col in df_leads.columns:
        df_leads[col] = pd.to_datetime(df_leads[col], errors="coerce")

# Handle the case where some date columns might not be present
# Only attempt to print if the columns are available
existing_date_columns = [col for col in date_columns if col in df_leads.columns]

# Print information and preview the data for available date columns
if existing_date_columns:
    print(df_leads[existing_date_columns].info())
    print(df_leads[existing_date_columns].head())
else:
    print("No date columns available for preview.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   fecha_ingreso_sistema  318 non-null    datetime64[ns]
 1   fecha                  318 non-null    datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 5.1 KB
None
  fecha_ingreso_sistema      fecha
0            2023-10-30 2023-10-30
1            2023-10-30 2023-10-30
2            2023-10-27 2023-10-27
3            2023-10-27 2023-10-27
4            2023-10-27 2023-10-27


In [79]:
# Verify the loaded data
print(df_leads.head())
print(df_leads.tail())
print(df_leads.dtypes)
print(df_leads.isnull().sum())

       fecha  dia  mes   año       regimen_aduanero              refrendo  \
0 2023-10-30   30   10  2023    importacion_courier  019-2023-91-01835123   
1 2023-10-30   30   10  2023  importacion_a_consumo  028-2023-10-01841170   
2 2023-10-27   27   10  2023  importacion_a_consumo  028-2023-10-01812140   
3 2023-10-27   27   10  2023  importacion_a_consumo  028-2023-10-01815066   
4 2023-10-27   27   10  2023  importacion_a_consumo  028-2023-10-01815066   

   item  ruc_importador                                probable_importador  \
0     1    992833831001                                     agenkacom_s.a.   
1    84    990792658001            importadora_lino_gamboa_cia._ltda._ilga   
2     2    993252182001  import_green_power_technology,_equipment_&_mac...   
3     9    992633980001                                         nadeu_s.a.   
4    10    992633980001                                         nadeu_s.a.   

  país_de_origen  ...                            dirección_consignat

In [80]:
# Step 1: Check if 'fecha' is already present and 'fecha_ingreso_sistema' exists
if 'fecha' not in df_leads.columns and 'fecha_ingreso_sistema' in df_leads.columns:
    # Rename 'fecha_ingreso_sistema' to 'fecha' and move it to the first position
    df_leads = df_leads.rename(columns={'fecha_ingreso_sistema': 'fecha'})
    # Reorder columns so that 'fecha' is the first column
    columns_order = ['fecha'] + [col for col in df_leads.columns if col != 'fecha']
    df_leads = df_leads[columns_order]
else:
    # If 'fecha' already exists, just drop 'fecha_ingreso_sistema' and move on
    if 'fecha_ingreso_sistema' in df_leads.columns:
        df_leads = df_leads.drop(columns='fecha_ingreso_sistema')

# Step 2: Drop the other date columns and 'mes', but only if they exist in the dataset
columns_to_drop = ['fecha_de_embarque', 'fecha_de_llegada', 'mes', 'dia', 'año']
columns_to_drop_existing = [col for col in columns_to_drop if col in df_leads.columns]

# Remove columns that exist in the dataset
df_leads = df_leads.drop(columns=columns_to_drop_existing)

# Step 3: Check the updated dataset
print(df_leads.head())
print(df_leads.shape)


       fecha       regimen_aduanero              refrendo  item  \
0 2023-10-30    importacion_courier  019-2023-91-01835123     1   
1 2023-10-30  importacion_a_consumo  028-2023-10-01841170    84   
2 2023-10-27  importacion_a_consumo  028-2023-10-01812140     2   
3 2023-10-27  importacion_a_consumo  028-2023-10-01815066     9   
4 2023-10-27  importacion_a_consumo  028-2023-10-01815066    10   

   ruc_importador                                probable_importador  \
0    992833831001                                     agenkacom_s.a.   
1    990792658001            importadora_lino_gamboa_cia._ltda._ilga   
2    993252182001  import_green_power_technology,_equipment_&_mac...   
3    992633980001                                         nadeu_s.a.   
4    992633980001                                         nadeu_s.a.   

  país_de_origen pais_de_procedencia ciudad_embarque via_de_transporte  ...  \
0          china               china     panama_city             aerea  ...   
1     

In [81]:
# Save the DataFrame as a CSV file
df_leads.to_csv('/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2023.csv', index=False)

print("File saved successfully.")


File saved successfully.


In [83]:
# Load the datasets
shape_2017 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2017.csv")
shape_2018 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2018.csv")
shape_2019 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2019.csv")
shape_2020 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2020.csv")
shape_2021 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2021.csv")
shape_2022 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2022.csv")
shape_2023 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/concat_2023.csv")

# List of DataFrames and their names for easier iteration
datasets = [
    ("2017", shape_2017),
    ("2018", shape_2018),
    ("2019", shape_2019),
    ("2020", shape_2020),
    ("2021", shape_2021),
    ("2022", shape_2022),
    ("2023", shape_2023)
]

# Columns to drop for years 2018-2023
columns_to_drop = ['agencia_de_carga', 'contenedor']

# Iterate through datasets to drop columns for 2018-2023 and print their shape and columns
for year, df in datasets:
    if year in ["2018", "2019", "2020", "2021", "2022", "2023"]:
        # Drop columns if they exist in the dataset
        df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    
    # Print dataset information
    print(f"Dataset for {year}:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}\n")


Dataset for 2017:
Shape: (291, 39)
Columns: ['fecha', 'regimen_aduanero', 'refrendo', 'item', 'ruc_importador', 'probable_importador', 'pais_de_origen', 'pais_de_procedencia', 'ciudad_embarque', 'via_de_transporte', 'aduana', 'partida_arancelaria', 'descripcion_arancelaria', 'descripcion_producto_comercial', 'marca', 'estado_de_mercancia', 'cantidad', 'unidad_de_medida', 'advalorem', 'us$_fob', 'us$_flete', 'us$_seguro', 'us$_cif', 'embarcador', 'empresa_de_transporte', 'pais_de_embarque', 'agente_de_aduana', 'nave', 'conocimiento_de_embarque', 'peso_neto_kg', 'tipo_aforo', 'dirección_consignatario', 'caracteristicas', 'producto', 'marca_comercial', 'año_fabricación', 'modelo_mercaderia', 'us$_fob_unit', 'código_tnan']

Dataset for 2018:
Shape: (318, 39)
Columns: ['fecha', 'regimen_aduanero', 'refrendo', 'item', 'ruc_importador', 'probable_importador', 'pais_de_origen', 'pais_de_procedencia', 'ciudad_embarque', 'via_de_transporte', 'aduana', 'partida_arancelaria', 'descripcion_arancela

In [84]:
# Ensure 'fecha' is a datetime column in all datasets for proper merging and sorting
for year, df in datasets:
    df['fecha'] = pd.to_datetime(df['fecha'])

# Merge the datasets on 'fecha'
df_merged = pd.concat([df for year, df in datasets], axis=0, ignore_index=True)

# Sort the merged dataset by 'fecha' from oldest to newest
df_merged = df_merged.sort_values(by='fecha').reset_index(drop=True)

# Save the merged and sorted dataset to a new CSV file
output_path = "/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2017-2023.csv"
df_merged.to_csv(output_path, index=False)

# Confirm successful save
print(f"Merged file saved to {output_path}")


Merged file saved to /home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2017-2023.csv


In [53]:
df_adjust = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2010-2012.csv")

if 'verifcador' in df_adjust.columns:
    df_adjust = df_adjust.drop(columns="verificador")
else:
    print("Column 'verifcador' not found.")

output_path = "/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2010-2012.csv"
df_adjust.to_csv(output_path, index=False)

print(df_adjust.shape)

Column 'verifcador' not found.
(173, 35)


In [85]:
# Load the narrowed down datasets
df_2010_2012 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2010-2012.csv")
df_2013_2016 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2013-2016.csv")
df_2017_2023 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2017-2023.csv")

# Display the shape and columns of each dataset
datasets_info = {
    "2010-2012": (df_2010_2012.shape, df_2010_2012.columns.tolist()),
    "2013-2016": (df_2013_2016.shape, df_2013_2016.columns.tolist()),
    "2017-2023": (df_2017_2023.shape, df_2017_2023.columns.tolist())
}

# Print dataset information
for year, (shape, columns) in datasets_info.items():
    print(f"Dataset for {year}:")
    print(f"Shape: {shape}")
    print(f"Columns: {columns}\n")


Dataset for 2010-2012:
Shape: (173, 35)
Columns: ['fecha', 'partida_arancelaria', 'producto', 'descripcion_arancelaria', 'ruc_importador', 'probable_importador', 'direccion', 'telefono', 'email1', 'refrendo', 'regimen_aduanero', 'pais_de_origen', 'pais_de_embarque', 'puerto_de_embarque', 'via_de_transporte', 'aduana', 'nave', 'empresa_de_transporte', 'agencia_de_transporte', 'dau', 'advalorem', 'unidades', 'peso_neto', 'peso_bruto', 'us$_fob', 'us$_flete', 'us$_seguro', 'us$_cif', 'bultos', 'tipo_de_bulto', 'estado_de_mercancia', 'embarcador', 'tipo_aforo', 'verificador', 'agente_de_aduana']

Dataset for 2013-2016:
Shape: (787, 34)
Columns: ['fecha', 'regimen_aduanero', 'refrendo', 'dau', 'item', 'ruc_importador', 'probable_importador', 'pais_de_origen', 'pais_de_procedencia', 'puerto_de_embarque', 'via_de_transporte', 'aduana', 'partida_arancelaria', 'descripcion_arancelaria', 'descripcion_producto_comercial', 'marca', 'estado_de_mercancia', 'bultos', 'unidades', 'tipo_de_bulto', 'adv

In [86]:
# Get the columns of each dataset
cols_2010_2012 = set(df_2010_2012.columns)
cols_2013_2016 = set(df_2013_2016.columns)
cols_2017_2023 = set(df_2017_2023.columns)

# Find columns present only in 2010-2012 and not in the other datasets
only_2010_2012 = cols_2010_2012 - cols_2013_2016 - cols_2017_2023

# Find columns present only in 2013-2016 and not in the other datasets
only_2013_2016 = cols_2013_2016 - cols_2010_2012 - cols_2017_2023

# Find columns present only in 2017-2023 and not in the other datasets
only_2017_2023 = cols_2017_2023 - cols_2010_2012 - cols_2013_2016

# Display the results
print("Columns present only in 2010-2012 and not in 2013-2016 or 2017-2023:")
print(only_2010_2012)
print("\n")

print("Columns present only in 2013-2016 and not in 2010-2012 or 2017-2023:")
print(only_2013_2016)
print("\n")

print("Columns present only in 2017-2023 and not in 2010-2012 or 2013-2016:")
print(only_2017_2023)


Columns present only in 2010-2012 and not in 2013-2016 or 2017-2023:
{'telefono', 'verificador', 'agencia_de_transporte', 'peso_bruto', 'email1', 'direccion'}


Columns present only in 2013-2016 and not in 2010-2012 or 2017-2023:
{'numero_de_manifiesto', 'container'}


Columns present only in 2017-2023 and not in 2010-2012 or 2013-2016:
{'ciudad_embarque', 'caracteristicas', 'descripción_arancelaria', 'país_de_origen', 'agencia_de_carga', 'modelo_mercaderia', 'marca_comercial', 'país_de_embarque', 'año_fabricación', 'unidad_de_medida', 'contenedor', 'dirección_consignatario', 'peso_neto_kg', 'código_tnan', 'us$_fob_unit', 'cantidad', 'conocimiento_embarque'}


In [87]:
# Find columns present in all three datasets
common_all_datasets = cols_2010_2012 & cols_2013_2016 & cols_2017_2023

# Display common columns across all datasets
if common_all_datasets:
    print("Columns present in all datasets (2010-2012, 2013-2016, 2017-2023):")
    print(common_all_datasets)
else:
    print("No columns found in all datasets.\n")

# Find columns present in 2013-2016 and not in 2017-2023
common_2013_2016_2017_2023 = cols_2013_2016 & cols_2017_2023
if common_2013_2016_2017_2023:
    print("Columns present in both 2013-2016 and 2017-2023:")
    print(common_2013_2016_2017_2023)
else:
    print("No common columns found between 2013-2016 and 2017-2023.\n")

# Find columns present in 2010-2012 and not in 2017-2023
common_2010_2012_2017_2023 = cols_2010_2012 & cols_2017_2023
if common_2010_2012_2017_2023:
    print("Columns present in both 2010-2012 and 2017-2023:")
    print(common_2010_2012_2017_2023)
else:
    print("No common columns found between 2010-2012 and 2017-2023.\n")

# Find columns present in 2010-2012 and not in 2013-2016
common_2010_2012_2013_2016 = cols_2010_2012 & cols_2013_2016
if common_2010_2012_2013_2016:
    print("Columns present in both 2010-2012 and 2013-2016:")
    print(common_2010_2012_2013_2016)
else:
    print("No common columns found between 2010-2012 and 2013-2016.\n")


Columns present in all datasets (2010-2012, 2013-2016, 2017-2023):
{'pais_de_embarque', 'fecha', 'descripcion_arancelaria', 'empresa_de_transporte', 'via_de_transporte', 'aduana', 'us$_fob', 'partida_arancelaria', 'pais_de_origen', 'refrendo', 'estado_de_mercancia', 'us$_flete', 'nave', 'probable_importador', 'us$_seguro', 'us$_cif', 'agente_de_aduana', 'embarcador', 'ruc_importador', 'advalorem', 'regimen_aduanero'}
Columns present in both 2013-2016 and 2017-2023:
{'pais_de_embarque', 'fecha', 'pais_de_procedencia', 'descripcion_arancelaria', 'empresa_de_transporte', 'via_de_transporte', 'aduana', 'us$_fob', 'partida_arancelaria', 'pais_de_origen', 'refrendo', 'item', 'marca', 'conocimiento_de_embarque', 'estado_de_mercancia', 'us$_flete', 'nave', 'probable_importador', 'us$_seguro', 'us$_cif', 'descripcion_producto_comercial', 'embarcador', 'agente_de_aduana', 'ruc_importador', 'advalorem', 'regimen_aduanero'}
Columns present in both 2010-2012 and 2017-2023:
{'pais_de_embarque', 'fec

In [89]:
import pandas as pd
import json

# Load datasets
df_2010_2012 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2010-2012.csv")
df_2013_2016 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2013-2016.csv")
df_2017_2023 = pd.read_csv("/home/luisvinatea/Data/Gdrive/BERAQUA/processed/prospectos_importadores/concat/merged/imp_aireadores_2017-2023.csv")

# List of columns to drop for each dataset
columns_to_drop_2010_2012 = ['telefono', 'agencia_de_transporte', 'email1', 'direccion', 'descripcion_arancelaria', 'pais_de_origen', 'agencia_de_carga']
columns_to_drop_2013_2016 = ['descripcion_arancelaria', 'pais_de_origen', 'agencia_de_carga']
columns_to_drop_2017_2023 = ['descripcion_arancelaria', 'pais_de_origen', 'agencia_de_carga']

# Rename columns as required
df_2010_2012 = df_2010_2012.rename(columns={'email1': 'email'})

# Initialize dictionary to store dropped column data
dropped_data = {}

# Create a helper function to store dropped columns in the dictionary
def store_dropped_columns(df, year_range):
    dropped_columns = {}
    
    # Check for each column before accessing
    for column in ['telefono', 'agencia_de_transporte', 'email', 'direccion', 'descripcion_arancelaria', 'pais_de_origen', 'agencia_de_carga', 'direccion_consignatario', 'modelo_mercaderia', 'marca_comercial', 'us$_fob']:
        if column in df.columns:
            dropped_columns[column] = df[column]
        else:
            dropped_columns[column] = None
    
    # Group by 'probable_importador' and aggregate values
    grouped_data = {}
    for key, values in dropped_columns.items():
        if values is not None:
            grouped_data[key] = df.groupby('probable_importador')[key].apply(list).to_dict()
    
    return grouped_data

# Apply the function to each dataset
dropped_data_2010_2012 = store_dropped_columns(df_2010_2012, "2010-2012")
dropped_data_2013_2016 = store_dropped_columns(df_2013_2016, "2013-2016")
dropped_data_2017_2023 = store_dropped_columns(df_2017_2023, "2017-2023")

# Combine all dropped data into a single dictionary
dropped_data.update(dropped_data_2010_2012)
dropped_data.update(dropped_data_2013_2016)
dropped_data.update(dropped_data_2017_2023)

# Drop columns from datasets (checking if the column exists before dropping)
df_2010_2012 = df_2010_2012.drop(columns=[col for col in columns_to_drop_2010_2012 if col in df_2010_2012.columns], errors='ignore')
df_2013_2016 = df_2013_2016.drop(columns=[col for col in columns_to_drop_2013_2016 if col in df_2013_2016.columns], errors='ignore')
df_2017_2023 = df_2017_2023.drop(columns=[col for col in columns_to_drop_2017_2023 if col in df_2017_2023.columns], errors='ignore')

# Merge all datasets into one
df_merged = pd.concat([df_2010_2012, df_2013_2016, df_2017_2023], axis=0, ignore_index=True)

# Sort by 'fecha'
df_merged['fecha'] = pd.to_datetime(df_merged['fecha'])
df_merged = df_merged.sort_values(by='fecha').reset_index(drop=True)

# Save the merged dataframe to CSV
output_path = "/home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/imp_aireadores_2010-2023.csv"
df_merged.to_csv(output_path, index=False)

# Save the dropped data to a JSON file
dropped_data_path = "/home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/product_data.json"
with open(dropped_data_path, 'w') as json_file:
    json.dump(dropped_data, json_file, indent=4)

# Confirm successful save
print(f"Merged file saved to {output_path}")
print(f"Dropped data saved to {dropped_data_path}")


Merged file saved to /home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/imp_aireadores_2010-2023.csv
Dropped data saved to /home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/product_data.json
